In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

# Choosing the product.
search_term = input("What do you want to search for?")

# Generating URL.
url = f"https://www.flipkart.com/search?q={search_term}&otracker=search&otracker1=search&marketplace=FLIPKART&as-" \
      f"show=on&as=off&as-pos=1&as-type=HISTORY"

# Making a get request.
result = requests.get(url)

# Parsing the HTML document.
doc = BeautifulSoup(result.text, "html.parser")

# Ensuring that the search term was valid.
empty = doc.find(class_="_3uTeW4")

if empty is None:

    # Counting the number of pages.
    doc_text = doc.find(class_="_2MImiq")
    contents = doc_text.span.string.split(" ")
    num = int(contents[-1])
    pages = 0

    # Limiting the number of pages to 50.
    if num > 50:
        pages = 50
    else:
        pages = num

    titles = []
    product_descriptions = []
    overall_ratings = []
    ratings = []
    reviews = []
    prices = []

    # Iterating through all the pages.
    for page in range(1, pages+1):
        url = f"https://www.flipkart.com/search?q={search_term}&otracker=search&otracker1=search&marketplace=" \
              f"FLIPKART&as-show=on&as=off&as-pos=1&as-type=HISTORY&page={page}"
        result = requests.get(url)
        doc = BeautifulSoup(result.text, "html.parser")

        # Getting the price.
        price_set = doc.find_all(class_="_30jeq3 _1_WHN1")
        for i in range(len(price_set)):
            prices.append(price_set[i].text)

        product = list(doc.find_all(class_="col col-7-12"))
        for i in range(len(product)):

            # Getting the title.
            title = product[i].find(class_="_4rR01T")
            titles.append(title.text)

            # Getting the description.
            description = product[i].find(class_="_1xgFaf")
            if description is not None:
                product_descriptions.append(description.text)
            else:
                product_descriptions.append(np.nan)

            # Getting the overall rating.
            overall_rating = product[i].find(class_="_3LWZlK")
            if overall_rating is not None:
                overall_ratings.append(overall_rating.text)
            else:
                overall_ratings.append(np.nan)

            # Getting the ratings and reviews.
            ratings_reviews = product[i].find(class_="_2_R_DZ")
            if ratings_reviews is not None:
                all_text = ratings_reviews.text.replace("&", "").replace("Ratings", "").replace("Reviews", "").\
                    replace(u'\xa0', u' ')
                ratings.append(all_text.split(" ")[0])
                reviews.append(all_text.split(" ")[3])
            else:
                ratings.append(np.nan)
                reviews.append(np.nan)

        keys = ["Title", "Description", "Overall Rating", "Ratings", "Reviews", "Price"]
        values = [titles, product_descriptions, overall_ratings, ratings, reviews, prices]

        product_dict = dict(zip(keys, values))
        
        # Creating a dataframe.
        df = pd.DataFrame.from_dict(product_dict)
        
        # Saving the dataframe to a csv.
        # df.to_csv(r"file path", index=False)
else:
    print("Invalid query.")

What do you want to search for?hhhhhhhhhhhhhhhhhhhhhhhhhhh
Invalid query.
